In [1]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
import pennylane as qml
from pennylane.templates import ApproxTimeEvolution
from numpy.linalg import solve

Here we provide the code related to Fig. 5 in the paper.

### Generate time evolution operator

In [ ]:
def Time_evolution_part(H,k,l,time,H_t):  
    if k>=l:
        time_evolution=qml.TrotterProduct(H,H_t+time*(k-l),5)
    else:
        time_evolution=qml.adjoint(qml.TrotterProduct(H,H_t+time*(k-l),5))
    return (time_evolution)

### Calculation of Real/Imaginary Parts in the novel Hadamard Test and the Approximate Inner Product function

In [ ]:
dev_re= qml.device('default.qubit', wires=11)
@qml.qnode(dev_re)
def hadamard_test_RE(H,k,l,time_step,Ht):
    for i in range (11):
        qml.Hadamard(i)
    qml.ControlledQubitUnitary(base=Time_evolution_part(H,k,l,time_step,Ht), control_wires=[10])
    qml.Hadamard(10)
    return (qml.expval(qml.PauliZ(10)))

dev_im= qml.device('default.qubit', wires=11)
@qml.qnode(dev_im)
def hadamard_test_IM(U,k,l,time_step,H_t):
    for i in range (11):
        qml.Hadamard(i)
    qml.PhaseShift(-np.pi/2,wires=10)
    qml.ControlledQubitUnitary(Time_evolution_part(H,k,l,time_step,H_t), control_wires=[10])
    qml.Hadamard(10)
    return (qml.expval(qml.PauliZ(10)))
def approximate_inner_product(k,l,H_t,time,H):
    IM_pp=-hadamard_test_IM(H,k,l,time,0)
    IM_pep=-hadamard_test_IM(H,k,l,time,H_t)
    RE_pp=hadamard_test_RE(H,k,l,time,0)
    RE_pep=hadamard_test_RE(H,k,l,time,H_t)
    IM_php=(RE_pep-RE_pp)
    RE_php=-IM_pep+IM_pp
    return ((-RE_php+1j*IM_php)/(H_t))

### Actual Inner Product Calculation

In [2]:
def actual_inner_product(k,l,matrix,vector,time_step):
    K=np.dot(expm(-1j*k*time_step*matrix),vector)
    L=np.dot(expm(-1j*l*time_step*matrix),vector)
    return(np.inner(L.T.conjugate(),np.dot(matrix,K)))

### 

### 论文中的一个例子

In [ ]:
%run ISING_INSPIRED_H.ipynb
H=Ising_inspired_Hamiltonian(10, 0.1,1.37,110.6982637623554)
U=qml.matrix(H)
def condition_number(matrix):
    eigv,eigc=np.linalg.eig(matrix)
    eigv
    abs_eigv=[]
    for i in eigv:
        abs_eigv.append(abs(i))
    return (max(abs_eigv)/min(abs_eigv),min(abs_eigv))
condition_number(U)

In [ ]:
def fidelity(k,l,H_t,time,H):
    b=np.ones((2**10,2**10))[0]/(np.sqrt(2**10))
    return(abs(approximate_inner_product(k,l,H_t,time,H)-actual_inner_product(k,l,qml.matrix(H),b,time)))

In [ ]:
Discrete_Time_Sequence=np.linspace(0.001, 0.1, num=20)
error=[]
for i in Discrete_Time_Sequence:
    error.append(fidelity(0,2,i,0.1,H))